<a href="https://colab.research.google.com/github/YoungsikMoon/FORS/blob/main/%EC%A0%95%ED%98%B8%EC%84%9D/FORS_%EC%A0%95%ED%98%B8%EC%84%9D_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_%EC%B5%9C%EC%A2%85%EC%84%9C%EB%B9%84%EC%8A%A4%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 최종 서비스 구현



#### Fast-Wshisper 모델 로딩

In [ ]:
import speech_recognition as sr
from gtts import gTTS
import os
import time
import pygame
import os
from datetime import datetime
from glob import glob
from faster_whisper import WhisperModel
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
import pandas as pd

In [ ]:
# 인텔 드라이버 충돌 에러
# 원래 있는데 다시 깔려고 하니까 중복 에러 뜸
# 중복돼도 OK에 True값 적용
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Fast Whisper 모델 불러오기
model = WhisperModel("large-v3")

#### 질문지 작성

In [ ]:
start_ment = """안녕하세요. 호석님. 서울시 중구 취업 센터 aI 상담사입니다.
시니어 취업 관련해서 간단한 사전 설문조사를 진행하고 있사오니, 잠깐 시간을 내어주시면 감사하겠습니다."""

question1 = "먼저 성함을 알려주시겠어요?"

question2 = "성별과 나이는 어떻게 되시나요?"

question3 = """건강상태를 간단히 여쭤보겠습니다.
걷는데는 지장이 없으신가요? 있으시다면 얼마만큼 불편하신가요?"""

question4 = "보고 듣는데는 지장이 없으신가요? 있으시다면 얼마만큼 불편하신가요?"

question5 = """학교는 어디까지 나오셨나요?
초등학교, 중학교, 고등학교, 대학교에 졸업한 곳으로 답변 부탁드립니다."""

question6 = "살면서 주로 무슨 일들을 하셨나요?"

question7 = "혹시 특별히 하고 싶은 일이 있으신가요?"

question8 = "배우자나 자녀분들은 어디에 살고 있나요?"

end_ment = """답변 감사드립니다. 서울시 중구 고용센터였습니다.
궁금하신 사항이 있으면 발송드린 문자에 있는 번호로 연락 부탁드립니다.
담당자 검토 후 수일 내로 다시 연락드리겠습니다. 감사합니다. """

# 질문 갯수
questions = [start_ment, question1, question2, question3, question4, question5, question6, question7, question8, end_ment]

# 답변 담을 리스트
answer = []

#### 필요 메소드 작성

In [ ]:

# 텍스트를 말로 바꿔줌
# param : 읽어줄 텍스트
def speak(text):
    tts = gTTS(text=text, lang='ko')
    filename='voice.mp3'
    tts.save(filename)
    pygame.init()
    pygame.mixer.music.load(filename)
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        continue
    pygame.mixer.music.unload()
    os.remove(filename)


# 마이크 음성 -> 오디오 파일로 저장 (구글이 만들어준 텍스트파일도 같이 저장)
# pram : 저장할 폴더 경로(str), 사용자정보(list), 질문번호(int)
def get_audio(folder_path, user_info, i):
    r = sr.Recognizer()
    with sr.Microphone() as source:

        print("지금 말씀하세요: ")

        # 오디오 파일로 저장
        audio = r.listen(source)
        wav_file_name = folder_path + '\\' + user_info[0] + "_" + user_info[1] + "_" + str(i)

        # 중복된거 있으면 그냥 지우고 다시 생성
        if os.path.isfile(wav_file_name + ".wav"):
            os.remove(wav_file_name + ".wav")

        try:
            with open(wav_file_name + ".wav", 'bx') as f:
                f.write(audio.get_wav_data())
        except Exception as e:
            print("Exception: " + str(e))

        # 구글에서 전사해준거 텍스트로 저장 (위스퍼랑 비교해보자)
        text = ""
        try:
            txt_file_name = folder_path + '\\' + user_info[0] + "_" + user_info[1] + "_google.txt"
            text = r.recognize_google(audio, language="ko-KR")
            print("말씀하신 내용입니다 : ", text)

            # 답변 전사된 텍스트 파일로 만들어 저장하기
            with open(txt_file_name, 'a') as f:
                f.write(str(text)+"\n")
        except Exception as e:
            print("Exception: " + str(e))

    return text


# #### 폴더 만들기 ####
# param : 만들 폴더 주소(str)
def make_folder(path: str):

    try:
        os.mkdir(path)

    # 혹시 이름 중복이면 일단 다른 폴더로 만들기
    # 해당 내용 로그 파일에 기록
    except:
        os.mkdir(path + "_Error")
        print(f"중복 폴더 존재{path}")


#### 녹음별 유저 폴더 만들기 ####
# param : 유저 정보 리스트(list)
# return : 만들어진 폴더 경로(str)
def make_user_voice_folder(user_info: list):

    try :
        date = datetime.today().strftime("%Y%m%d")
        num = user_info[1]
        name = user_info[0]

        # 상위 폴더는 일단 날짜명. 없으면 만들어줌
        if os.path.exists(f"03.음성녹음\\{date}") == False:
            os.mkdir("03.음성녹음\\" + date)

        # 폴더가 없다면
        folder_path = "03.음성녹음\\" + date + "\\" + num + "_" + name
        if os.path.exists(folder_path) == False:

            # 날짜 폴더 밑에 [식별번호_이름] 폴더를 만들어줌
            make_folder(folder_path)

    # 혹시 중복일 경우 일단 다른 폴더를 만들어주고 로그 남김
    except Exception as e:
        print("Exception: " + str(e))

    return folder_path

# 위스퍼로 오디오파일 전사하기
# param : 폴더경로(str)
# return : 답변 텍스트 리스트(list)
def whisper_transcribe(folder_path):

    answer_lst = []
    for audio_file in sorted(glob(folder_path + "/*.wav")):
        segments, info = model.transcribe(audio_file)
        for segment in segments:
            answer_lst.append(segment.text)

    return answer_lst


# GPT야 요약해줘
# param : 답변 내용 리스트(list)
# return : 요약내용 리스트(list)
def get_gpt_help(answer_lst):
    # 재우 API
    GPT_API_KEY = "sk-proj-KgZChuKTiML1O6WHDO7oT3BlbkFJvJYPMkdQyzZ3v4cc9XML"

    # API 키로 LLM 객체 생성 (GPT와 연결해줌)
    # temperature : 생성된 텍스트의 다양성 조정
    # 0~2 사이
    # 높을 수록 출력을 무작위하게, 낮을 수록 출력을 더 집중되고 결정론적으로 만듦
    # model_name : 사용할 GPT 모델(버전) 정보
    # openai_api_key : API 키값
    chat = ChatOpenAI(temperature = 0.2, max_tokens = 2024, openai_api_key = GPT_API_KEY)

    system_msg = "이 글에서 '이c름, 나이, 건강상태, 최종학력, 경력사항, 희망사항, 가족사항' 으로 요약해줘"
    human_msg = ".".join(answer_lst)


    # 시스템 메세지로 원하는 답변의 형태를 지정할 수 있음
    messages = [
        SystemMessage(content=system_msg),
        HumanMessage(content=human_msg),
    ]

    # 질문하고 답받아서 잘라서 리스트로 만들기
    gpt_re_lst = str(chat.invoke(messages).content).split('\n')

    return gpt_re_lst



#### 메인 서비스 코드 작성

In [ ]:
# 유저 폴더 만들기 및 경로 추출
user_info = ["001234001213", "김옥자", "", ""]
folder_path = make_user_voice_folder(user_info)



for i, question in enumerate(questions):

    # 질문하기
    speak(question)

    # 첫 번째랑 마지막은 안내멘트로 답변 안받음
    if 0 < i < len(questions) - 1:

        # 답변받고 파일 저장(오디오파일, 텍스트파일)
        get_audio(folder_path, user_info, i)

# 혹시 테스트하다가 중간에 그냥 끊으면 플레이어 언로드해주기
pygame.mixer.music.unload

# 위스퍼로 해당 폴더 오디오 내용 전사하기
answer_lst = whisper_transcribe(folder_path)

# 랭체인 GPT로 요약해서 받기
gpt_re_lst = get_gpt_help(answer_lst)


# 데이터프레임으로 만들기

# 답변 내용만 가져와서 리스트로 만들기
# 검증 코드가 필요하겠지만.. 여기까지만...
# 실제로 사용한다면 DB에다가 저장
content = []
for sentence in gpt_re_lst:
    content.append(sentence.split(": ")[1])

result_df = pd.DataFrame(columns =["이름","나이","건강상태","최종학력","경력사항","희망사항","가족사항"])
result_df.loc[len(result_df)] = content


In [ ]:
# 결과 저장 데이터프레임
result_df

In [ ]:
# 결과 엑셀로 추출
result_df.to_excel("조사결과.xlsx", index = False)